The goal of this notebook is to prove the theorem here: /home/ubuntu/ohm-tree-filesys/plasma-converter/logs/minif2f-valid-one-step_2024-10-18_19-23-18

```------------------------------------Problem-------------------------------------
/-- Determine the value of $ab$ if $\log_8a+\log_4b^2=5$ and $\log_8b+\log_4a^2=7$. Show that it is 512.-/
theorem aime_1984_p5 (a b : ℝ) (h₀ : Real.logb 8 a + Real.logb 4 (b ^ 2) = 5)
  (h₁ : Real.logb 8 b + Real.logb 4 (a ^ 2) = 7) : a * b = 512 := by```

In [1]:
from typing import Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model = AutoModelForCausalLM = AutoModelForCausalLM.from_pretrained(
    'deepseek-ai/DeepSeek-Prover-V1.5-RL',
    trust_remote_code=True,
    device_map='auto'
)

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.7k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    'deepseek-ai/DeepSeek-Prover-V1.5-RL',
    trust_remote_code=True
)

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

In [4]:
tokenizer.pad_token = tokenizer.eos_token

In [5]:
def tokenize(prompt: str) -> Tuple[torch.Tensor, torch.Tensor]:
        tokens = tokenizer(
            prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")
        return tokens['input_ids'], tokens['attention_mask']

In [6]:
def complete(input_ids, attention_mask, max_length=1000):
    return model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=input_ids.shape[1] + max_length
    )

In [7]:
prompt = """/-- Determine the value of $ab$ if $\log_8a+\log_4b^2=5$ and $\log_8b+\log_4a^2=7$. Show that it is 512.-/
theorem aime_1984_p5 (a b : ℝ) (h₀ : Real.logb 8 a + Real.logb 4 (b ^ 2) = 5)
  (h₁ : Real.logb 8 b + Real.logb 4 (a ^ 2) = 7) : a * b = 512 := by"""

In [8]:
input_ids, attention_mask = tokenize(prompt)
output_tokens = complete(input_ids, attention_mask)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [9]:
completed_proof = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [10]:
print(completed_proof)

/-- Determine the value of $ab$ if $\log_8a+\log_4b^2=5$ and $\log_8b+\log_4a^2=7$. Show that it is 512.-/
theorem aime_1984_p5 (a b : ℝ) (h₀ : Real.logb 8 a + Real.logb 4 (b ^ 2) = 5)
  (h₁ : Real.logb 8 b + Real.logb 4 (a ^ 2) = 7) : a * b = 512 := by
  have h₂ : a > 0 := by
    have h₂ := h₀
    have h₃ := h₁
    linarith [Real.logb_nonneg 8 a, Real.logb_nonneg 4 (b ^ 2), Real.logb_nonneg 8 b,
      Real.logb_nonneg 4 (a ^ 2)]
  have h₃ : b > 0 := by
    have h₂ := h₀
    have h₃ := h₁
    linarith [Real.logb_nonneg 8 a, Real.logb_nonneg 4 (b ^ 2), Real.logb_nonneg 8 b,
      Real.logb_nonneg 4 (a ^ 2)]
  field_simp [Real.logb, Real.log_rpow] at h₀ h₁
  linarith
```


In [11]:
def prove(prompt):
    input_ids, attention_mask = tokenize(prompt)
    output_tokens = complete(input_ids, attention_mask)
    completed_proofs = [tokenizer.decode(output_tokens[i], skip_special_tokens=True) for i in range(len(output_tokens))]
    return completed_proofs

In [12]:
prompt_comments = """Complete the following Lean 4 code with explanatory comments.
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat 
/-- Determine the value of $ab$ if $\log_8a+\log_4b^2=5$ and $\log_8b+\log_4a^2=7$. Show that it is 512.-/
theorem aime_1984_p5 (a b : ℝ) (h₀ : Real.logb 8 a + Real.logb 4 (b ^ 2) = 5)
  (h₁ : Real.logb 8 b + Real.logb 4 (a ^ 2) = 7) : a * b = 512 := by
  """

In [13]:
proosf = prove(prompt_comments)
print(proofs[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['Complete the following Lean 4 code with explanatory comments.\nimport Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat \n/-- Determine the value of $ab$ if $\\log_8a+\\log_4b^2=5$ and $\\log_8b+\\log_4a^2=7$. Show that it is 512.-/\ntheorem aime_1984_p5 (a b : ℝ) (h₀ : Real.logb 8 a + Real.logb 4 (b ^ 2) = 5)\n  (h₁ : Real.logb 8 b + Real.logb 4 (a ^ 2) = 7) : a * b = 512 := by\n  \n  /-\n  Given the equations \\(\\log_8 a + \\log_4 b^2 = 5\\) and \\(\\log_8 b + \\log_4 a^2 = 7\\), we need to determine the value of \\(ab\\).\n  \n  1. Rewrite the equations using properties of logarithms:\n     \\[\n     \\log_8 a + 2 \\log_4 b = 5\n     \\]\n     \\[\n     \\log_8 b + 2 \\log_4 a = 7\n     \\]\n  \n  2. Convert the logarithms to base 2:\n     \\[\n     \\log_8 a = \\frac{\\log_2 a}{\\log_2 8} = \\frac{\\log_2 a}{3}\n     \\]\n     \\[\n     \\log_4 b = \\frac{\\log_2 b}{\\log_2 4} = \\frac{\\log_2 b}{2}\n     \\]\n  \n  3. Substitute thes

In [24]:
prompt_summarize = """```import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Simplify $(a-1)(a+1)(a+2) - (a-2)(a+1).$ Show that it is a^3 + a^2.-/
theorem mathd_algebra_568 (a : ℝ) :
    (a - 1) * (a + 1) * (a + 2) - (a - 2) * (a + 1) = a ^ 3 + a ^ 2 := by
  -- Expand and simplify the expression using algebraic rules.
  ring_nf```

Summarize this lean proof in English:
"""

In [25]:
proofs = prove(prompt_summarize)
print(proofs)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['```import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n/-- Simplify $(a-1)(a+1)(a+2) - (a-2)(a+1).$ Show that it is a^3 + a^2.-/\ntheorem mathd_algebra_568 (a : ℝ) :\n    (a - 1) * (a + 1) * (a + 2) - (a - 2) * (a + 1) = a ^ 3 + a ^ 2 := by\n  -- Expand and simplify the expression using algebraic rules.\n  ring_nf```\n\nSummarize this lean proof in English:\nThe theorem states that for any real number \\( a \\), the expression \\((a-1)(a+1)(a+2) - (a-2)(a+1)\\) simplifies to \\( a^3 + a^2 \\). The proof uses the `ring_nf` tactic to expand and simplify the expression, applying algebraic rules to achieve the desired form.']


In [26]:
print(proofs[0])

```import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- Simplify $(a-1)(a+1)(a+2) - (a-2)(a+1).$ Show that it is a^3 + a^2.-/
theorem mathd_algebra_568 (a : ℝ) :
    (a - 1) * (a + 1) * (a + 2) - (a - 2) * (a + 1) = a ^ 3 + a ^ 2 := by
  -- Expand and simplify the expression using algebraic rules.
  ring_nf```

Summarize this lean proof in English:
The theorem states that for any real number \( a \), the expression \((a-1)(a+1)(a+2) - (a-2)(a+1)\) simplifies to \( a^3 + a^2 \). The proof uses the `ring_nf` tactic to expand and simplify the expression, applying algebraic rules to achieve the desired form.
